In [1]:
import numpy as np
import pandas as pd
import nltk
from fuzzywuzzy import fuzz
from numpy import save
import itertools

/home/ig5eq/.local/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
ndc = pd.read_csv("../data/working/ndc_clean.csv", index_col = [0])

In [3]:
ndc.dropna(inplace = True)
ndc.reset_index(inplace = True)

In [4]:
ndc_row = ndc['row']

In [5]:
ndc_org = ndc['original_company']

In [6]:
ndc_clean = ndc['cleaned_name']

In [7]:
dna = pd.read_csv("../data/working/dna_clean.csv", index_col = [0])

In [8]:
dna.dropna(inplace = True)
dna.reset_index(inplace = True)

In [9]:
dna.columns

Index(['index', 'Unnamed: 0.1', 'Code', 'Description', 'cleaned_companies'], dtype='object')

In [10]:
dna_row = dna['index']

In [11]:
dna_org = dna['Description']

In [12]:
dna_clean = dna['cleaned_companies']

### Matching across NDC and DNA

In [13]:
##Function that takes in 3 lists, the list of strings you want to compare, the list of strings you're comparing to and a list with the original way the string from list 1 was written.  
def matchWithFuzzyNamesOG(l1, l2, og1, og2, row_num1, row_num2):
    match = pd.DataFrame(columns = ['clean NDC company', 'original NDC company', 'NDC row', 'clean DNA row', 'clean DNA company', 'original DNA company', 'fuzz ratio'])
    for (i, o, r) in zip(l1, og1, row_num1):
        
        matches = []
        score = []
        #score_partial = []
        row = []
        matchesog = []
        for (j, q, s) in zip(l2, og2, row_num2):
            if (fuzz.ratio(i,j)) == 100:
                matches.append(j)
                matchesog.append(q)
                score.append(fuzz.ratio(i,j))
                row.append(s)
                #the partial ratio will return the ratio raw score of 100  if the shorter string is found an any of substrings of the larger string, used this as a safety net and potentially as an additional step to 
               # score_partial.append((fuzz.partial_ratio(i,j)))
                match = match.append({ 'clean NDC company': i, 'original NDC company': o, 'NDC row': r, 'clean DNA row': row, 'clean DNA company': matches,   'original DNA company': matchesog, 'fuzz ratio': score,},ignore_index=True)
        
#Will output a pandas dataframe with 4 columns
    return match

In [ ]:
ndcxdna_match = matchWithFuzzyNamesOG(ndc_clean, dna_clean, ndc_org, dna_org, ndc_row, dna_row)

In [ ]:
ndcxdna_match

In [ ]:
ndcxdna_match = ndcxdna_match[~ndcxdna_match.duplicated('NDC row')]

In [ ]:
ndcxdna_match

In [ ]:
ndcxdna_match['clean NDC company'] = ndcxdna_match['clean NDC company'].astype(str)
ndcxdna_match['original NDC company'] = ndcxdna_match['original NDC company'].astype(str)
ndcxdna_match['NDC row'] = ndcxdna_match['NDC row'].astype(str)

In [ ]:
ndcxdna_match_unique = ndcxdna_match.groupby('clean NDC company', as_index = False).agg({'original NDC company': '^ '.join, 
                                                                            'NDC row': '^ '.join
                                                                                })

In [ ]:
ndcxdna_match_unique

In [ ]:
ndcxdna_match1 = ndcxdna_match.groupby('clean NDC company', as_index = False).first()

In [ ]:
ndcxdna_match_unique[['corporate family', 'clean DNA row', 'clean DNA company', 'original DNA company', 'fuzz ratio']] = ndcxdna_match1[['clean NDC company','clean DNA row', 'clean DNA company', 'original DNA company', 'fuzz ratio']]

In [ ]:
ndcxdna_match_unique.to_csv('../data/working/ndc_dna_matching.csv')